In [522]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient
from pprint import pprint
import time

## Get Table of Cities in CA

In [542]:
url = "https://www.california-demographics.com/cities_by_population"

In [543]:
df_pandas = pd.read_html(url, attrs = {'class': 'ranklist'},  flavor='bs4')

In [544]:
city = df_pandas[0]
city = city[:-1]  # drop last row

In [545]:
city.loc[:,'Population'] = pd.to_numeric(city.loc[:,'Population'])

C:\Users\mtp22\anaconda3\lib\site-packages\pandas\core\indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [546]:
city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360 entries, 0 to 1359
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rank        1360 non-null   object
 1   City        1360 non-null   object
 2   Population  1360 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 32.0+ KB


In [547]:
CA_City = city.loc[:,'City']
CA_City_list = CA_City.tolist()

## Get Lat Long for each City

In [262]:
token = 'be222025311da0ce8928afbea7d32547'
domain = 'http://api.positionstack.com/v1/forward'
long = []
lat = []

for i in range(len(CA_City_list)):
    params = {
        'access_key' : token,
        'query' : CA_City_list[i] + ',California' }
    
    response = requests.get(domain, params = params)
    raw_data = response.json()
    
    for j in raw_data:
        if j == "error":
            lat.append('NaN')
            long.append('NaN')
            
        elif len(raw_data['data']) == 0:
            lat.append('NaN')
            long.append('NaN')
        
        else:
            dict = raw_data['data']
            lat.append(dict[0]['latitude'])
            long.append(dict[0]['longitude'])

In [538]:
CA_location = pd.DataFrame({
    "City": CA_City_list,
    "Latitude": lat,
    "Longitude": long,
    "Population" : city['Population']
})

CA_location

,City,Latitude,Longitude,Population
0,Los Angeles,34.05513,-118.25703,3898747
1,San Diego,32.72793,-117.15529,1386932
2,San Jose,37.354559,-121.883844,1013240
3,San Francisco,37.778008,-122.431272,873965
4,Fresno,36.75156,-119.800612,542107
...,...,...,...,...
1355,Poso Park,35.811151,-118.636536,9
1356,Pearsonville and Warner Valley,40.398015,-121.318455,8
1357,Sugarloaf Village,35.826855,-118.63598,7
1358,Almanor,40.214481,-121.176019,3


In [282]:
# data cleaning - exclude cities that are not in California 
CA_location = CA_location[CA_location.Latitude != 'NaN'].reset_index(drop=True)
CA_location.loc[:,'Latitude'] = pd.to_numeric(CA_location.loc[:,'Latitude'])
CA_location.loc[:,'Longitude'] = pd.to_numeric(CA_location.loc[:,'Longitude'])
CA_Table = CA_location[(CA_location.Latitude >= 32.573145) & (CA_location.Latitude <= 41.595985) & (CA_location.Longitude >= -124.40) & (CA_location.Longitude <= -114.21)].reset_index(drop=True)

In [539]:
CA_Table

,City,Latitude,Longitude,Population,GridX,GridY,Forecast Office,API_Call
0,Los Angeles,34.055130,-118.257030,3898747,154,44,LOX,"https://api.weather.gov/gridpoints/LOX/154,44/..."
1,San Diego,32.727930,-117.155290,1386932,56,13,SGX,"https://api.weather.gov/gridpoints/SGX/56,13/f..."
2,San Jose,37.354559,-121.883844,1013240,100,83,MTR,"https://api.weather.gov/gridpoints/MTR/100,83/..."
3,San Francisco,37.778008,-122.431272,873965,84,105,MTR,"https://api.weather.gov/gridpoints/MTR/84,105/..."
4,Fresno,36.751560,-119.800612,542107,52,100,HNX,"https://api.weather.gov/gridpoints/HNX/52,100/..."
...,...,...,...,...,...,...,...,...
1310,Poso Park,35.811151,-118.636536,9,85,51,HNX,"https://api.weather.gov/gridpoints/HNX/85,51/f..."
1311,Pearsonville and Warner Valley,40.398015,-121.318455,8,62,147,STO,"https://api.weather.gov/gridpoints/STO/62,147/..."
1312,Sugarloaf Village,35.826855,-118.635980,7,86,52,HNX,"https://api.weather.gov/gridpoints/HNX/86,52/f..."
1313,Almanor,40.214481,-121.176019,3,65,138,STO,"https://api.weather.gov/gridpoints/STO/65,138/..."


## Weather Forecast API

### Get Grid Points / Forecast Office for each city

In [298]:
gridX = []
gridY = []
weather_domain = 'https://api.weather.gov/points/'

for i in range(len(CA_Table)):
    a = str(CA_Table['Latitude'][i])
    b = str(CA_Table['Longitude'][i])
    response = requests.get(weather_domain + a + ',' + b)
    forecast = response.json()
    
    if len(forecast) == 5:
        gridX.append(forecast['properties']['gridX'])
        gridY.append(forecast['properties']['gridY'])
        office.append(forecast['properties']['cwa'])
    else:
        gridX.append('NaN')
        gridY.append('NaN')
        office.append('NaN')
    

In [497]:
CA_Table['GridX'] = gridX
CA_Table['GridY'] = gridY
CA_Table['Forecast Office'] = office

# forecast_domain = 'https://api.weather.gov/gridpoints/'
# api_call = []
# for i in range(len(CA_Table)):
#     api_call.append(forecast_domain + str(CA_Table['Forecast Office'][i]) + '/' + str(CA_Table['GridX'][i]) + ',' + str(CA_Table['GridY'][i]) + '/forecast')

# CA_Table['API_Call'] = api_call 


In [507]:
CA_Table.info()
# CA_Table.to_csv(r'C:\Users\mtp22\OneDrive\文件\Python Scripts\CA_Table.csv', index = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             1315 non-null   object 
 1   Latitude         1315 non-null   float64
 2   Longitude        1315 non-null   float64
 3   Population       1315 non-null   int64  
 4   GridX            1315 non-null   int64  
 5   GridY            1315 non-null   int64  
 6   Forecast Office  1315 non-null   object 
 7   API_Call         1315 non-null   object 
dtypes: float64(2), int64(3), object(3)
memory usage: 82.3+ KB


### Get forecast from Grid Points

In [501]:
city_name = CA_Table['City']

In [524]:
temp_forecast = []
for i in range(len(city_name)):
    dict = {}
    dict['City'] = city_name[i]
    response = requests.get(forecast_domain + str(CA_Table['Forecast Office'][i]) + '/' + str(CA_Table['GridX'][i]) + ',' + str(CA_Table['GridY'][i]) + '/forecast')
#     print(response.json())
#     print(response.url())
    if len(response.json()) == 6:
        dict['Status'] = 'Nan'
    else:
        seven_day = response.json()['properties']['periods']
        
        for j in range(len(seven_day)):
            dict[str(seven_day[j]['name'])] = seven_day[j]['temperature']

        temp_forecast.append(dict)
        
    print(dict)


{'City': 'Los Angeles', 'This Afternoon': 75, 'Tonight': 52, 'Wednesday': 80, 'Wednesday Night': 54, 'Thursday': 78, 'Thursday Night': 53, 'Friday': 75, 'Friday Night': 53, 'Saturday': 68, 'Saturday Night': 49, 'Sunday': 66, 'Sunday Night': 48, 'Monday': 70, 'Monday Night': 51}
{'City': 'San Diego', 'This Afternoon': 73, 'Tonight': 53, 'Wednesday': 69, 'Wednesday Night': 51, 'Thursday': 75, 'Thursday Night': 53, 'Friday': 74, 'Friday Night': 53, 'Saturday': 69, 'Saturday Night': 52, 'Sunday': 63, 'Sunday Night': 48, 'Monday': 70, 'Monday Night': 51}
{'City': 'San Jose', 'This Afternoon': 68, 'Tonight': 42, 'Wednesday': 70, 'Wednesday Night': 44, 'Thursday': 67, 'Thursday Night': 44, 'Friday': 71, 'Friday Night': 45, 'Saturday': 61, 'Saturday Night': 39, 'Sunday': 61, 'Sunday Night': 40, 'Monday': 69, 'Monday Night': 46}
{'City': 'San Francisco', 'This Afternoon': 61, 'Tonight': 48, 'Wednesday': 62, 'Wednesday Night': 49, 'Thursday': 60, 'Thursday Night': 48, 'Friday': 62, 'Friday Night

{'City': 'Garden Grove', 'This Afternoon': 76, 'Tonight': 51, 'Wednesday': 75, 'Wednesday Night': 51, 'Thursday': 79, 'Thursday Night': 51, 'Friday': 78, 'Friday Night': 52, 'Saturday': 72, 'Saturday Night': 50, 'Sunday': 68, 'Sunday Night': 46, 'Monday': 75, 'Monday Night': 51}
{'City': 'Palmdale', 'This Afternoon': 76, 'Tonight': 45, 'Wednesday': 71, 'Wednesday Night': 43, 'Thursday': 72, 'Thursday Night': 45, 'Friday': 75, 'Friday Night': 47, 'Saturday': 68, 'Saturday Night': 40, 'Sunday': 56, 'Sunday Night': 36, 'Monday': 63, 'Monday Night': 42}
{'City': 'Salinas', 'This Afternoon': 67, 'Tonight': 43, 'Wednesday': 66, 'Wednesday Night': 44, 'Thursday': 66, 'Thursday Night': 43, 'Friday': 69, 'Friday Night': 46, 'Saturday': 61, 'Saturday Night': 40, 'Sunday': 60, 'Sunday Night': 40, 'Monday': 66, 'Monday Night': 46}
{'City': 'Hayward', 'This Afternoon': 64, 'Tonight': 45, 'Wednesday': 65, 'Wednesday Night': 46, 'Thursday': 63, 'Thursday Night': 46, 'Friday': 66, 'Friday Night': 47, 

{'City': 'Temecula', 'Status': 'Nan'}
{'City': 'Santa Maria', 'Status': 'Nan'}
{'City': 'West Covina', 'This Afternoon': 80, 'Tonight': 53, 'Wednesday': 78, 'Wednesday Night': 54, 'Thursday': 80, 'Thursday Night': 53, 'Friday': 80, 'Friday Night': 54, 'Saturday': 70, 'Saturday Night': 48, 'Sunday': 67, 'Sunday Night': 46, 'Monday': 73, 'Monday Night': 50}
{'City': 'El Monte', 'This Afternoon': 81, 'Tonight': 51, 'Wednesday': 80, 'Wednesday Night': 51, 'Thursday': 83, 'Thursday Night': 52, 'Friday': 81, 'Friday Night': 53, 'Saturday': 72, 'Saturday Night': 47, 'Sunday': 70, 'Sunday Night': 46, 'Monday': 77, 'Monday Night': 50}
{'City': 'Inglewood', 'This Afternoon': 71, 'Tonight': 55, 'Wednesday': 79, 'Wednesday Night': 54, 'Thursday': 78, 'Thursday Night': 53, 'Friday': 76, 'Friday Night': 53, 'Saturday': 69, 'Saturday Night': 50, 'Sunday': 68, 'Sunday Night': 50, 'Monday': 74, 'Monday Night': 52}
{'City': 'Burbank', 'Status': 'Nan'}
{'City': 'El Cajon', 'This Afternoon': 80, 'Tonight'

{'City': 'Lake Forest', 'This Afternoon': 78, 'Tonight': 51, 'Wednesday': 75, 'Wednesday Night': 53, 'Thursday': 80, 'Thursday Night': 54, 'Friday': 79, 'Friday Night': 54, 'Saturday': 70, 'Saturday Night': 50, 'Sunday': 65, 'Sunday Night': 46, 'Monday': 72, 'Monday Night': 52}
{'City': 'Newport Beach', 'This Afternoon': 70, 'Tonight': 51, 'Wednesday': 69, 'Wednesday Night': 51, 'Thursday': 74, 'Thursday Night': 54, 'Friday': 71, 'Friday Night': 54, 'Saturday': 67, 'Saturday Night': 52, 'Sunday': 66, 'Sunday Night': 50, 'Monday': 70, 'Monday Night': 53}
{'City': 'San Ramon', 'This Afternoon': 66, 'Tonight': 39, 'Wednesday': 69, 'Wednesday Night': 42, 'Thursday': 65, 'Thursday Night': 41, 'Friday': 69, 'Friday Night': 44, 'Saturday': 59, 'Saturday Night': 36, 'Sunday': 61, 'Sunday Night': 37, 'Monday': 69, 'Monday Night': 45}
{'City': 'Redwood City', 'Status': 'Nan'}
{'City': 'Buena Park', 'This Afternoon': 79, 'Tonight': 52, 'Wednesday': 77, 'Wednesday Night': 52, 'Thursday': 81, 'Thur

{'City': 'Yuba City', 'This Afternoon': 43, 'Tonight': 35, 'Thursday': 48, 'Thursday Night': 34, 'Friday': 47, 'Friday Night': 31, "New Year's Day": 46, 'Saturday Night': 31, 'Sunday': 47, 'Sunday Night': 36, 'Monday': 49, 'Monday Night': 45, 'Tuesday': 50, 'Tuesday Night': 45}
{'City': 'Eastvale', 'This Afternoon': 83, 'Tonight': 49, 'Wednesday': 78, 'Wednesday Night': 50, 'Thursday': 83, 'Thursday Night': 50, 'Friday': 83, 'Friday Night': 52, 'Saturday': 77, 'Saturday Night': 48, 'Sunday': 68, 'Sunday Night': 45, 'Monday': 77, 'Monday Night': 50}
{'City': 'Tulare', 'This Afternoon': 72, 'Tonight': 45, 'Wednesday': 70, 'Wednesday Night': 43, 'Thursday': 72, 'Thursday Night': 42, 'Friday': 74, 'Friday Night': 46, 'Saturday': 67, 'Saturday Night': 42, 'Sunday': 61, 'Sunday Night': 37, 'Monday': 68, 'Monday Night': 43}
{'City': 'Palo Alto', 'This Afternoon': 66, 'Tonight': 44, 'Wednesday': 66, 'Wednesday Night': 44, 'Thursday': 64, 'Thursday Night': 44, 'Friday': 68, 'Friday Night': 46, 

...

{'City': 'Crows Landing', 'Status': 'Nan'}
{'City': 'Philo', 'This Afternoon': 62, 'Tonight': 34, 'Wednesday': 63, 'Wednesday Night': 37, 'Thursday': 62, 'Thursday Night': 36, 'Friday': 66, 'Friday Night': 39, 'Saturday': 56, 'Saturday Night': 32, 'Sunday': 60, 'Sunday Night': 34, 'Monday': 68, 'Monday Night': 40}
{'City': 'Rail Road Flat and Oak Shores', 'This Afternoon': 66, 'Tonight': 37, 'Wednesday': 69, 'Wednesday Night': 38, 'Thursday': 70, 'Thursday Night': 40, 'Friday': 71, 'Friday Night': 43, 'Saturday': 61, 'Saturday Night': 36, 'Sunday': 61, 'Sunday Night': 37, 'Monday': 69, 'Monday Night': 42}
{'City': 'Victor', 'Status': 'Nan'}
{'City': 'Richfield and Princeton', 'Status': 'Nan'}
{'City': 'Trinidad', 'This Afternoon': 52, 'Tonight': 32, 'Wednesday': 60, 'Wednesday Night': 41, 'Thursday': 57, 'Thursday Night': 38, 'Friday': 61, 'Friday Night': 39, 'Saturday': 46, 'Saturday Night': 29, 'Sunday': 52, 'Sunday Night': 33, 'Monday': 63, 'Monday Night': 41}
{'City': 'Honcut', 'Th

{'City': 'Snelling', 'Status': 'Nan'}
{'City': 'Proberta, Moskowite Corner, Herlong, and Moss Landing', 'This Afternoon': 64, 'Tonight': 42, 'Wednesday': 62, 'Wednesday Night': 41, 'Thursday': 62, 'Thursday Night': 40, 'Friday': 64, 'Friday Night': 44, 'Saturday': 59, 'Saturday Night': 39, 'Sunday': 62, 'Sunday Night': 40, 'Monday': 67, 'Monday Night': 44}
{'City': 'Sierra City', 'This Afternoon': 54, 'Tonight': 35, 'Wednesday': 61, 'Wednesday Night': 38, 'Thursday': 60, 'Thursday Night': 36, 'Friday': 65, 'Friday Night': 39, 'Saturday': 52, 'Saturday Night': 29, 'Sunday': 52, 'Sunday Night': 32, 'Monday': 62, 'Monday Night': 40}
{'City': 'Richvale', 'This Afternoon': 67, 'Tonight': 46, 'Wednesday': 72, 'Wednesday Night': 46, 'Thursday': 68, 'Thursday Night': 42, 'Friday': 73, 'Friday Night': 48, 'Saturday': 61, 'Saturday Night': 40, 'Sunday': 67, 'Sunday Night': 42, 'Monday': 77, 'Monday Night': 50}
{'City': 'Bombay Beach', 'This Afternoon': 85, 'Tonight': 58, 'Wednesday': 85, 'Wednes

{'City': 'Verdi', 'Status': 'Nan'}
{'City': 'Swall Meadows', 'Status': 'Nan'}
{'City': 'Tupman', 'This Afternoon': 74, 'Tonight': 48, 'Wednesday': 71, 'Wednesday Night': 45, 'Thursday': 75, 'Thursday Night': 45, 'Friday': 76, 'Friday Night': 48, 'Saturday': 72, 'Saturday Night': 42, 'Sunday': 61, 'Sunday Night': 40, 'Monday': 69, 'Monday Night': 44}
{'City': 'Montgomery Creek and Nicolaus', 'This Afternoon': 69, 'Tonight': 43, 'Wednesday': 72, 'Wednesday Night': 44, 'Thursday': 68, 'Thursday Night': 40, 'Friday': 72, 'Friday Night': 46, 'Saturday': 62, 'Saturday Night': 40, 'Sunday': 66, 'Sunday Night': 40, 'Monday': 75, 'Monday Night': 47}
{'City': 'Paradise CDP', 'Today': 39, 'Tonight': 30, 'Tuesday': 36, 'Tuesday Night': 33, 'Wednesday': 39, 'Wednesday Night': 32, 'Thursday': 43, 'Thursday Night': 35, 'Friday': 43, 'Friday Night': 30, "New Year's Day": 42, 'Saturday Night': 32, 'Sunday': 41, 'Sunday Night': 38}
{'City': 'Timber Cove', 'Status': 'Nan'}
{'City': 'Franklin CDP and Comp

{'City': 'Camp Nelson', 'This Afternoon': 44, 'Tonight': 33, 'Wednesday': 43, 'Wednesday Night': 35, 'Thursday': 44, 'Thursday Night': 35, 'Friday': 48, 'Friday Night': 37, 'Saturday': 42, 'Saturday Night': 28, 'Sunday': 32, 'Sunday Night': 27, 'Monday': 43, 'Monday Night': 35}
{'City': 'Volcano', 'This Afternoon': 88, 'Tonight': 53, 'Wednesday': 88, 'Wednesday Night': 55, 'Thursday': 85, 'Thursday Night': 51, 'Friday': 87, 'Friday Night': 53, 'Saturday': 88, 'Saturday Night': 51, 'Sunday': 75, 'Sunday Night': 49, 'Monday': 80, 'Monday Night': 51}
{'City': 'Washington', 'Tonight': 42, 'Monday': 48, 'Monday Night': 33, 'Tuesday': 41, 'Tuesday Night': 38, 'Wednesday': 42, 'Wednesday Night': 33, 'Thursday': 46, 'Thursday Night': 34, 'Friday': 46, 'Friday Night': 33, "New Year's Day": 45, 'Saturday Night': 32, 'Sunday': 45}
{'City': 'Creston', 'This Afternoon': 70, 'Tonight': 42, 'Wednesday': 72, 'Wednesday Night': 41, 'Thursday': 71, 'Thursday Night': 41, 'Friday': 75, 'Friday Night': 46,

{'City': 'Fellows', 'This Afternoon': 69, 'Tonight': 50, 'Wednesday': 67, 'Wednesday Night': 49, 'Thursday': 71, 'Thursday Night': 49, 'Friday': 72, 'Friday Night': 53, 'Saturday': 69, 'Saturday Night': 45, 'Sunday': 58, 'Sunday Night': 45, 'Monday': 65, 'Monday Night': 48}
{'City': 'Ponderosa and Keddie', 'Status': 'Nan'}
{'City': 'California Hot Springs', 'This Afternoon': 56, 'Tonight': 41, 'Wednesday': 53, 'Wednesday Night': 44, 'Thursday': 58, 'Thursday Night': 43, 'Friday': 61, 'Friday Night': 47, 'Saturday': 55, 'Saturday Night': 37, 'Sunday': 42, 'Sunday Night': 35, 'Monday': 54, 'Monday Night': 43}
{'City': 'Eagleville, McGee Creek, and Randsburg', 'This Afternoon': 75, 'Tonight': 49, 'Wednesday': 72, 'Wednesday Night': 49, 'Thursday': 70, 'Thursday Night': 49, 'Friday': 74, 'Friday Night': 51, 'Saturday': 71, 'Saturday Night': 46, 'Sunday': 61, 'Sunday Night': 43, 'Monday': 69, 'Monday Night': 50}
{'City': 'Panorama Heights and Sattley', 'This Afternoon': 49, 'Tonight': 28, '

In [548]:
short_forecast = []
for i in range(len(city_name)):
    dict = {}
    dict['City'] = city_name[i]
    response = requests.get(forecast_domain + str(CA_Table['Forecast Office'][i]) + '/' + str(CA_Table['GridX'][i]) + ',' + str(CA_Table['GridY'][i]) + '/forecast')
#     print(response.json())
#     print(response.url())
    if len(response.json()) == 6:
        dict['Status'] = 'Nan'
    else:
        seven_day = response.json()['properties']['periods']
        
        for j in range(len(seven_day)):
            dict[str(seven_day[j]['name'])] = seven_day[j]['shortForecast']

        short_forecast.append(dict)
        
    print(dict)

{'City': 'Los Angeles', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Partly Sunny then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Slight Chance Light Rain then Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'San Diego', 'Tonight': 'Patchy Fog', 'Wednesday': 'Patchy Fog then Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Patchy Fog', 'Friday': 'Patchy Fog then Sunny', 'Friday Night': 'Patchy Fog', 'Saturday': 'Patchy Fog then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'San Jose', 'This Afternoon': 'Mostly Sunny', 'Toni

{'City': 'Moreno Valley', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Partly Sunny then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Slight Chance Light Rain', 'Sunday Night': 'Partly Cloudy', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Fontana', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Partly Sunny then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Oxnard', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clea

{'City': 'Escondido', 'Tonight': 'Partly Cloudy then Patchy Fog', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy then Patchy Fog', 'Saturday': 'Patchy Fog then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Roseville', 'Tonight': 'Patchy Fog', 'Wednesday': 'Patchy Fog then Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Patchy Fog then Mostly Sunny', 'Friday Night': 'Rain Showers Likely', 'Saturday': 'Rain Showers', 'Saturday Night': 'Chance Rain Showers then Partly Cloudy', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Torrance', 'Tonight': 'Patchy Fog', 'Wednesday':

{'City': 'Antioch', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Mostly Sunny', 'Friday Night': 'Light Rain Likely', 'Saturday': 'Light Rain', 'Saturday Night': 'Chance Light Rain then Partly Cloudy', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Carlsbad', 'Tonight': 'Patchy Fog', 'Wednesday': 'Patchy Fog then Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy then Patchy Fog', 'Saturday': 'Patchy Fog then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Downey', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Nigh

{'City': 'Hesperia', 'Status': 'Nan'}
{'City': 'Vista', 'Tonight': 'Partly Cloudy then Patchy Fog', 'Wednesday': 'Patchy Fog then Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy then Patchy Fog', 'Saturday': 'Patchy Fog then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Compton', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Partly Sunny then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Slight Chance Light Rain then Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Cars

...

{'City': 'Verdi', 'Tonight': 'Mostly Cloudy', 'Wednesday': 'Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Mostly Sunny', 'Friday Night': 'Mostly Cloudy then Chance Rain And Snow', 'Saturday': 'Rain And Snow Likely', 'Saturday Night': 'Chance Rain And Snow then Mostly Cloudy', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Swall Meadows', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Light Snow Likely', 'Saturday Night': 'Chance Light Snow', 'Sunday': 'Slight Chance Light Snow then Mostly Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Tupman', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Mostly

{'City': 'Valley Ford', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Mostly Sunny', 'Friday Night': 'Light Rain', 'Saturday': 'Light Rain', 'Saturday Night': 'Mostly Clear', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Milford and Casmalia', 'Tonight': 'Patchy Fog', 'Wednesday': 'Patchy Fog then Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Patchy Fog', 'Saturday': 'Chance Light Rain', 'Saturday Night': 'Chance Light Rain then Partly Cloudy', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Whitehawk', 'Tonight': 'Mostly Cloudy', 'Wednesday': 'Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 

{'City': 'Washington', 'Tonight': 'Light Rain', 'Monday': 'Chance Light Rain then Partly Sunny', 'Monday Night': 'Areas Of Frost then Slight Chance Rain And Snow', 'Tuesday': 'Chance Rain And Snow', 'Tuesday Night': 'Chance Light Rain', 'Wednesday': 'Chance Light Rain', 'Wednesday Night': 'Mostly Cloudy then Areas Of Frost', 'Thursday': 'Areas Of Frost then Mostly Sunny', 'Thursday Night': 'Slight Chance Light Rain', 'Friday': 'Slight Chance Light Rain then Partly Sunny', 'Friday Night': 'Mostly Clear', "New Year's Day": 'Mostly Sunny', 'Saturday Night': 'Partly Cloudy', 'Sunday': 'Slight Chance Light Rain'}
{'City': 'Creston', 'Status': 'Nan'}
{'City': 'El Rancho', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Mostly Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Light Rain Likely', 'Saturday Night': 'Light Rain Likely', 'Sunday': 'Slight Chance Light Rain then Partly

{'City': 'Old Station', 'Tonight': 'Areas Of Fog', 'Wednesday': 'Patchy Fog then Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Partly Cloudy', 'Friday': 'Mostly Sunny', 'Friday Night': 'Slight Chance Rain Showers then Rain And Snow Showers Likely', 'Saturday': 'Rain And Snow Showers', 'Saturday Night': 'Chance Rain And Snow Showers then Partly Cloudy', 'Sunday': 'Mostly Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Mostly Sunny', 'Monday Night': 'Partly Cloudy', 'Tuesday': 'Sunny'}
{'City': 'Tennant', 'Tonight': 'Mostly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Cloudy', 'Thursday': 'Mostly Cloudy', 'Thursday Night': 'Partly Cloudy', 'Friday': 'Partly Sunny', 'Friday Night': 'Chance Light Rain then Rain And Snow', 'Saturday': 'Rain And Snow', 'Saturday Night': 'Slight Chance Rain And Snow Showers then Mostly Cloudy', 'Sunday': 'Mostly Sunny', 'Sunday Night': 'Partly Cloudy', 'Monday': 'Mostly Sunny', 'Monday Night': 'Part

{'City': 'Prattville and Sequoia Crest', 'Status': 'Nan'}
{'City': 'Posey', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Mostly Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Chance Light Rain', 'Saturday Night': 'Chance Light Rain then Chance Rain And Snow', 'Sunday': 'Chance Rain And Snow then Mostly Cloudy', 'Sunday Night': 'Partly Cloudy', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'City': 'Shoshone, Johnsville, and Bucks Lake', 'Tonight': 'Mostly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Partly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Mostly Sunny', 'Friday Night': 'Chance Rain And Snow Showers', 'Saturday': 'Snow Showers', 'Saturday Night': 'Chance Snow Showers then Partly Cloudy', 'Sunday': 'Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'T

## DB Creation

In [447]:
client = MongoClient('mongodb://localhost:27017')
db=client.admin
serverStatusResult=db.command("serverStatus")
pprint(serverStatusResult)

{'asserts': {'msg': 0,
             'regular': 0,
             'rollovers': 0,
             'tripwire': 0,
             'user': 1324,
             'warning': 0},
 'catalogStats': {'capped': 0,
                  'collections': 17,
                  'internalCollections': 3,
                  'internalViews': 0,
                  'timeseries': 0,
                  'views': 0},
 'connections': {'active': 2,
                 'available': 999988,
                 'awaitingTopologyChanges': 1,
                 'current': 12,
                 'exhaustHello': 1,
                 'exhaustIsMaster': 0,
                 'threaded': 12,
                 'totalCreated': 286},
 'electionMetrics': {'averageCatchUpOps': 0.0,
                     'catchUpTakeover': {'called': 0, 'successful': 0},
                     'electionTimeout': {'called': 0, 'successful': 0},
                     'freezeTimeout': {'called': 0, 'successful': 0},
                     'numCatchUps': 0,
                     'numCat

In [454]:
myclient = MongoClient("mongodb://localhost:27017")
mydb = myclient["Week_Forecast"]
mycol = mydb.get_collection("Temperature")

In [455]:
mycol.insert_many(temp_forecast)

In [529]:
mydoc = mycol.find({})
for i in mydoc:
    print(i)

{'_id': ObjectId('6230f924708d8d3c8b140e15'), 'City': 'Los Angeles', 'This Afternoon': 75, 'Tonight': 52, 'Wednesday': 80, 'Wednesday Night': 54, 'Thursday': 78, 'Thursday Night': 53, 'Friday': 75, 'Friday Night': 53, 'Saturday': 68, 'Saturday Night': 49, 'Sunday': 66, 'Sunday Night': 48, 'Monday': 70, 'Monday Night': 51}
{'_id': ObjectId('6230f924708d8d3c8b140e16'), 'City': 'San Diego', 'This Afternoon': 73, 'Tonight': 52, 'Wednesday': 70, 'Wednesday Night': 51, 'Thursday': 76, 'Thursday Night': 53, 'Friday': 74, 'Friday Night': 53, 'Saturday': 68, 'Saturday Night': 52, 'Sunday': 62, 'Sunday Night': 48, 'Monday': 68, 'Monday Night': 51}
{'_id': ObjectId('6230f924708d8d3c8b140e17'), 'City': 'San Jose', 'This Afternoon': 68, 'Tonight': 42, 'Wednesday': 70, 'Wednesday Night': 44, 'Thursday': 67, 'Thursday Night': 44, 'Friday': 71, 'Friday Night': 45, 'Saturday': 61, 'Saturday Night': 39, 'Sunday': 61, 'Sunday Night': 40, 'Monday': 69, 'Monday Night': 46}
{'_id': ObjectId('6230f924708d8d3

In [549]:
mycol2 = mydb.get_collection("ShortForecast")

In [550]:
mycol2.insert_many(short_forecast)

In [553]:
mydoc2 = mycol2.find({})

for i in mydoc2:
    print(i)

{'_id': ObjectId('623144e7708d8d3c8b141253'), 'City': 'Los Angeles', 'Tonight': 'Partly Cloudy', 'Wednesday': 'Sunny', 'Wednesday Night': 'Mostly Clear', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Mostly Clear', 'Friday': 'Sunny', 'Friday Night': 'Partly Cloudy', 'Saturday': 'Partly Sunny then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Slight Chance Light Rain then Sunny', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostly Clear', 'Tuesday': 'Sunny'}
{'_id': ObjectId('623144e7708d8d3c8b141254'), 'City': 'San Diego', 'Tonight': 'Patchy Fog', 'Wednesday': 'Patchy Fog then Mostly Sunny', 'Wednesday Night': 'Partly Cloudy', 'Thursday': 'Mostly Sunny', 'Thursday Night': 'Patchy Fog', 'Friday': 'Patchy Fog then Sunny', 'Friday Night': 'Patchy Fog', 'Saturday': 'Patchy Fog then Chance Light Rain', 'Saturday Night': 'Chance Light Rain', 'Sunday': 'Chance Light Rain', 'Sunday Night': 'Mostly Clear', 'Monday': 'Sunny', 'Monday Night': 'Mostl

# *Ignore Below*

In [388]:
# temp_forecast = []
# desc_forecast = []

# for i in seven_day:
#     dict = {}
# #     time = i['name']
# #     temperature = i['temperature']
# #     windspeed = i['windSpeed']
# #     desc = i['shortForecast']
#     dict = {
#         'Time': i['name'],
#         'Temperature': i['temperature'],
#     }
    
#     temp_forecast.append(dict)

In [389]:
# temp_forecast

[{'Time': 'Today', 'Temperature': 75},
 {'Time': 'Tonight', 'Temperature': 52},
 {'Time': 'Wednesday', 'Temperature': 80},
 {'Time': 'Wednesday Night', 'Temperature': 54},
 {'Time': 'Thursday', 'Temperature': 78},
 {'Time': 'Thursday Night', 'Temperature': 53},
 {'Time': 'Friday', 'Temperature': 75},
 {'Time': 'Friday Night', 'Temperature': 53},
 {'Time': 'Saturday', 'Temperature': 68},
 {'Time': 'Saturday Night', 'Temperature': 49},
 {'Time': 'Sunday', 'Temperature': 66},
 {'Time': 'Sunday Night', 'Temperature': 48},
 {'Time': 'Monday', 'Temperature': 70},
 {'Time': 'Monday Night', 'Temperature': 51}]

In [368]:
# for j in seven_day:
#     dict2 = {}
#     dict2 = {
#         'Time': j['name'],
#         'Short Description': j['shortForecast'],
#     }
    
#     desc_forecast.append(dict2)

In [412]:
# forecast_domain = 'https://api.weather.gov/gridpoints/'
# str(CA_Table['Forecast Office'][16])
# response = requests.get(forecast_domain + str(CA_Table['Forecast Office'][16]) + '/' + str(CA_Table['GridX'][16]) + ',' + str(CA_Table['GridY'][16]) + '/forecast')    
# seven_day = response.json()['properties']['periods']
# city_name = CA_Table['City']

In [413]:
# temp_forecast = []
# dict = {}
# for i in seven_day:
#     dict['City'] = city_name[16]
#     dict[str(i['name'])] = i['temperature']


In [414]:
# dict

{'City': 'Santa Clarita',
 'Today': 79,
 'Tonight': 48,
 'Wednesday': 76,
 'Wednesday Night': 49,
 'Thursday': 79,
 'Thursday Night': 49,
 'Friday': 77,
 'Friday Night': 49,
 'Saturday': 70,
 'Saturday Night': 43,
 'Sunday': 60,
 'Sunday Night': 42,
 'Monday': 68,
 'Monday Night': 46}

In [476]:
# gridX2 = []
# gridY2 = []
# office2 = []
# url = []
# weather_domain2 = 'https://api.weather.gov/points/'
# forecast_domain = 'https://api.weather.gov/gridpoints/'


# for i in range(len(CA_Table)):
#     a = str(CA_Table['Latitude'][i])
#     b = str(CA_Table['Longitude'][i])
#     response2 = requests.get(weather_domain2 + a + ',' + b)
#     forecast2 = response2.json()
    
#     if len(forecast2) == 5:
#         gridX2.append(forecast2['properties']['gridX'])
#         gridY2.append(forecast2['properties']['gridY'])
#         office2.append(forecast2['properties']['cwa'])
#         url.append(response2.url)
#     else:
#         gridX2.append('NaN')
#         gridY2.append('NaN')
#         office2.append('NaN')

In [530]:
# api_call = []
# CA_Directory = pd.DataFrame({
#     "City": CA_Table['City'],
#     "Office": office2,
#     "Grid X": gridX2,
#     "Grid Y": gridY2,
# })

# for i in range(len(CA_Directory)):
#     api_call.append(forecast_domain + str(CA_Directory['Office'][i]) + '/' + str(CA_Directory['Grid X'][i]) + ',' + str(CA_Directory['Grid Y'][i]) + '/forecast')

# CA_Directory['API_Call'] = api_call 

# CA_Directory.to_csv(r'C:\Users\mtp22\OneDrive\文件\Python Scripts\CA_Directory.csv', index = False)

In [531]:
# for i in range(30):
#     dict = {}
#     dict['City'] = city_name[i]
#     response = requests.get(api_call[i])
#     print(response.url)
#     if len(response.json()) == 6:
#         dict['Status'] = 'Nan'
#     else:
#         seven_day = response.json()['properties']['periods']
        
#         for j in range(len(seven_day)):
#             dict[str(seven_day[j]['name'])] = seven_day[j]['temperature']

#         temp_forecast.append(dict)
        
#         time.sleep(2)
        
#     print(dict)

https://api.weather.gov/gridpoints/LOX/154,44/forecast
{'City': 'Los Angeles', 'This Afternoon': 75, 'Tonight': 52, 'Wednesday': 80, 'Wednesday Night': 54, 'Thursday': 78, 'Thursday Night': 53, 'Friday': 75, 'Friday Night': 53, 'Saturday': 68, 'Saturday Night': 49, 'Sunday': 66, 'Sunday Night': 48, 'Monday': 70, 'Monday Night': 51}
https://api.weather.gov/gridpoints/SGX/56,13/forecast
{'City': 'San Diego', 'This Afternoon': 73, 'Tonight': 53, 'Wednesday': 69, 'Wednesday Night': 51, 'Thursday': 75, 'Thursday Night': 53, 'Friday': 74, 'Friday Night': 53, 'Saturday': 69, 'Saturday Night': 52, 'Sunday': 63, 'Sunday Night': 48, 'Monday': 70, 'Monday Night': 51}
https://api.weather.gov/gridpoints/MTR/100,83/forecast
{'City': 'San Jose', 'This Afternoon': 68, 'Tonight': 42, 'Wednesday': 70, 'Wednesday Night': 44, 'Thursday': 67, 'Thursday Night': 44, 'Friday': 71, 'Friday Night': 45, 'Saturday': 61, 'Saturday Night': 39, 'Sunday': 61, 'Sunday Night': 40, 'Monday': 69, 'Monday Night': 46}
http

{'City': 'Elk Grove', 'Overnight': 42, 'Monday': 49, 'Monday Night': 33, 'Tuesday': 43, 'Tuesday Night': 37, 'Wednesday': 44, 'Wednesday Night': 34, 'Thursday': 47, 'Thursday Night': 34, 'Friday': 47, 'Friday Night': 31, "New Year's Day": 45, 'Saturday Night': 30, 'Sunday': 47}
https://api.weather.gov/gridpoints/SGX/49,73/forecast
{'City': 'Ontario', 'Tonight': 42, 'Friday': 58, 'Friday Night': 40, "New Year's Day": 58, 'Saturday Night': 37, 'Sunday': 60, 'Sunday Night': 38, 'Monday': 59, 'Monday Night': 40, 'Tuesday': 59, 'Tuesday Night': 42, 'Wednesday': 63, 'Wednesday Night': 44, 'Thursday': 67}
https://api.weather.gov/gridpoints/SGX/51,77/forecast
{'City': 'Rancho Cucamonga', 'This Afternoon': 80, 'Tonight': 52, 'Wednesday': 77, 'Wednesday Night': 54, 'Thursday': 80, 'Thursday Night': 54, 'Friday': 81, 'Friday Night': 56, 'Saturday': 76, 'Saturday Night': 50, 'Sunday': 65, 'Sunday Night': 47, 'Monday': 75, 'Monday Night': 54}
https://api.weather.gov/gridpoints/SGX/54,36/forecast
{'

In [481]:
# temp_forecast = []
# forecast_domain = 'https://api.weather.gov/gridpoints/'
# for i in range(50):
#     dict = {}
#     dict['City'] = city_name[i]
#     response = requests.get(CA_Directory['API'][i])
#     print(response.json())
#     if len(response.json()) == 6:
#         dict['Status'] = 'Nan'
#     else:
#         seven_day = response.json()['properties']['periods']
        
#         for j in range(len(seven_day)):
#             dict[str(seven_day[j]['name'])] = seven_day[j]['temperature']

#         temp_forecast.append(dict)
        
#     print(dict)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld', {'@version': '1.1', 'wx': 'https://api.weather.gov/ontology#', 's': 'https://schema.org/', 'geo': 'http://www.opengis.net/ont/geosparql#', 'unit': 'http://codes.wmo.int/common/unit/', '@vocab': 'https://api.weather.gov/ontology#', 'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'}, 'city': 's:addressLocality', 'state': 's:addressRegion', 'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'}, 'bearing': {'@type': 's:QuantitativeValue'}, 'value': {'@id': 's:value'}, 'unitCode': {'@id': 's:unitCode', '@type': '@id'}, 'forecastOffice': {'@type': '@id'}, 'forecastGridData': {'@type': '@id'}, 'publicZone': {'@type': '@id'}, 'county': {'@type': '@id'}}], 'id': 'https://api.weather.gov/points/34.0551,-118.257', 'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.257, 34.0551]}, 'properties': {'@id': 'https://api.weather.gov/points/34.0551,-118.257', '@type': 'wx:Point', 'cwa': '

KeyError: 'periods'

In [306]:
# office = []

# for i in range(len(CA_Table)):
#     a = str(CA_Table['Latitude'][i])
#     b = str(CA_Table['Longitude'][i])
#     response = requests.get(weather_domain + a + ',' + b)
#     forecast = response.json()
    
#     if len(forecast) == 5:
#         office.append(forecast['properties']['cwa'])
#     else:
#         office.append('NaN')